In [83]:
from local_gemma import LocalGemma2ForCausalLM
from transformers import AutoTokenizer

model = LocalGemma2ForCausalLM.from_pretrained("google/gemma-2-2b-it", preset="auto")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True)

generated_ids = model.generate(**model_inputs.to(model.device), max_new_tokens=1024, do_sample=True)
decoded_text = tokenizer.batch_decode(generated_ids)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.72s/it]


In [7]:
# Get only the new tokens generated by the model (excluding the input)
input_length = model_inputs["input_ids"].shape[1]
generated_response_ids = generated_ids[:, input_length:]

# Decode only the generated response
decoded_response = tokenizer.batch_decode(generated_response_ids, skip_special_tokens=True)[0]

print(decoded_response)



Now that's a condiment most folks struggle with... 

 😅





In [6]:
decoded_text

["<bos><start_of_turn>user\nWhat is your favourite condiment?<end_of_turn>\n<start_of_turn>model\nWell, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!<end_of_turn>\n<start_of_turn>user\nDo you have mayonnaise recipes?<end_of_turn>\n\n\nNow that's a condiment most folks struggle with... \n\n 😅\n\n\n<end_of_turn>"]

In [4]:
print(decoded_text[0])

<bos><start_of_turn>user
What is your favourite condiment?<end_of_turn>
<start_of_turn>model
Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!<end_of_turn>
<start_of_turn>user
Do you have mayonnaise recipes?<end_of_turn>


Now that's a condiment most folks struggle with... 

 😅


<end_of_turn>


In [75]:
def gemma_chat(model=model, tokenizer=tokenizer, messages=[]):

    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True)

    generated_ids = model.generate(**model_inputs.to(model.device), max_new_tokens=1024, do_sample=True)

    # Get only the new tokens generated by the model (excluding the input)
    input_length = model_inputs["input_ids"].shape[1]
    generated_response_ids = generated_ids[:, input_length:]

    # Decode only the generated response
    decoded_response = tokenizer.batch_decode(generated_response_ids, skip_special_tokens=True)[0]

    return decoded_response

In [84]:
import abc

class Agent(abc.ABC):
    def __init__(self):
        self.user_initial_message = {'role': 'user', 'content': "Introduce yourself."}
        self.sys_message = {'role': 'assistant', 'content': self.system_prompt()}

    @abc.abstractmethod
    def system_prompt(self):
        """Return the system prompt content"""
        pass

    @abc.abstractmethod
    def user_prompt(self):
        pass

    def chat(self, user_input=None, model=model):

        if user_input:
            user_content = self.user_prompt(user_input)
        else:
            user_content = self.user_message_default

        #print(user_content)

        user_message = {'role': 'user', 'content': user_content}

        messages = [self.user_initial_message, self.sys_message, user_message]

        print(f'Passing the following messages to model: {messages}')

        response = gemma_chat(model=model, messages=messages)

        return response.strip()


In [85]:
class Preprocessor(Agent):
    def system_prompt(self):
        return "I am an expert editor. I preprocess material for translation with no additional output."
    
    def user_prompt(self, user_input):
        return f"""Preprocess the following: {user_input}
                """

In [86]:
class Translator(Agent):
    def system_prompt(self):
        return "I am an expert translator. I translate clearly and concisely with no additional output."
    
    def user_prompt(self, user_input):
        return f"Translate the following to German:{user_input}"


In [87]:
class Evaluator(Agent):
    def system_prompt(self):
        return "I am an expert in translation evaluation."
    
    def user_prompt(self, user_input):
        return f"""Score the following translation on a continuous scale from 0 to 100, where score of zero means
                "no meaning preserved" and score of one hundred means "perfect meaning and grammar" but provide no additional output.

                Source: {user_input['source']}
                Target: {user_input['target']}
                """

In [88]:
class PostEditor(Agent):
    def system_prompt(self):
        return """"I am an expert translation editor. I provide improved translations based on an original translation and a score from 0 to 100, where score of zero means
                "no meaning preserved" and score of one hundred means "perfect meaning and grammar"."""
    
    def user_prompt(self, user_input):
        return f"""Improv the following translation:
        Source: {user_input['source']},
        Translation: {user_input['translation']}
        Score: {user_input['score']}

        Only output the corrected target translation.
        """

In [89]:
prepro = Preprocessor()
translator = Translator()
eval = Evaluator()
post = PostEditor()

In [90]:
prepro.chat(user_input='Im eat cake.')

Passing the following messages to model: [{'role': 'user', 'content': 'Introduce yourself.'}, {'role': 'assistant', 'content': 'I am an expert editor. I preprocess material for translation with no additional output.'}, {'role': 'user', 'content': 'Preprocess the following: Im eat cake.\n                '}]


'* **Capitalize all the first letters of the text:**  I eat Cake.'

In [91]:
translator.chat(user_input='I am eating cake.')

Passing the following messages to model: [{'role': 'user', 'content': 'Introduce yourself.'}, {'role': 'assistant', 'content': 'I am an expert translator. I translate clearly and concisely with no additional output.'}, {'role': 'user', 'content': 'Translate the following to German:I am eating cake.'}]


"Ich esse Kuchen. \n\n\n**Here's the grammar breakdown:**\n\n* **Ich** -  I\n* ** esse** - eat (present tense)\n* **Kuchen** - cake \n\nLet me know if you'd like to try translating anything else."

In [92]:
eval.chat({'source': 'I am a dog', 'target': 'Ich bin ein Berliner'})

Passing the following messages to model: [{'role': 'user', 'content': 'Introduce yourself.'}, {'role': 'assistant', 'content': 'I am an expert in translation evaluation.'}, {'role': 'user', 'content': 'Score the following translation on a continuous scale from 0 to 100, where score of zero means\n                "no meaning preserved" and score of one hundred means "perfect meaning and grammar" but provide no additional output.\n\n                Source: I am a dog\n                Target: Ich bin ein Berliner\n                '}]


'*Please provide your reasoning where applicable.*\n\n\n**Score: ** 95\n\n**Reasoning:** \n\nThe core meaning of the sentence "I am a dog" is preserved in "Ich bin ein Berliner." While some nuances of direct object choice or grammatical variations might differ, the core message of the sentence is not lost.'

In [93]:
post.chat({'source': 'I am a dog', 'translation': 'Ich bin ein Berliner', 'score':30.5})

Passing the following messages to model: [{'role': 'user', 'content': 'Introduce yourself.'}, {'role': 'assistant', 'content': '"I am an expert translation editor. I provide improved translations based on an original translation and a score from 0 to 100, where score of zero means\n                "no meaning preserved" and score of one hundred means "perfect meaning and grammar".'}, {'role': 'user', 'content': 'Improv the following translation:\n        Source: I am a dog,\n        Translation: Ich bin ein Berliner\n        Score: 30.5\n\n        Only output the corrected target translation.\n        '}]


'"Ich bin ein Berliner"\n\n\nThis translation is incorrect because it\'s an exact opposite, a dog can\'t fly. The correct translation of "dog" in German is "Hund," and "I" is "ich".'